<a href="https://colab.research.google.com/github/YunusKaratepe/duygu-tanima/blob/main/BP_CNN_ML_Full_%C4%B0lker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [105]:
import keras
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras import activations

# Load Data From Google Drive

In [106]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
!ls
!pwd

drive  sample_data
/content


# Get Data Set

In [108]:
def getDataset(): 
  directory = '/content/drive/My Drive/Bp_dataset_v4/MelSpectrogram_128/MelSpectrogramOriginal_Mel128'

  train_set=tf.keras.preprocessing.image_dataset_from_directory(
      directory,
      labels="inferred",
      label_mode="categorical",
      subset="training",
      class_names=None,
      shuffle=True,
      seed = 100, #10 50 100 verecegiz 
      batch_size=32,
      image_size=(128, 128),
      validation_split = 0.2,
      #interpolation="lanczos5"
  )

  test_set = tf.keras.preprocessing.image_dataset_from_directory(
      directory,
      labels="inferred",
      label_mode="categorical",
      subset="validation",
      validation_split = 0.2,
      class_names=None,
      shuffle=True,
      seed = 100, #10 50 100 verecegiz
      batch_size=32,
      image_size=(128, 128),
      #interpolation="lanczos5"
  )
  
  #print(test_set)
  #for data, labels in test_set:
    #print(data.shape)  # (64, 200, 200, 3)
  return train_set, test_set

# Training Custom CNN

In [109]:
def trainCustomCNN(train_set, test_set):
  X_train  = train_set
  opt  = tf.keras.optimizers.Adam(learning_rate=0.00001)

  #input_shape=(128, 128),
  model = keras.Sequential() 
  
  model.add(keras.layers.Conv2D(128, input_shape=(128,128,3), kernel_size=80, strides=4, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))

  model.add(keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))

  model.add(keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))

  model.add(keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', kernel_initializer='glorot_uniform', kernel_regularizer=keras.regularizers.l2(l=0.0001)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('relu'))
  model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))


  model.add(keras.layers.Lambda(lambda x : keras.backend.mean(x, axis=1)))
  model.add(keras.layers.Lambda(lambda x : keras.backend.mean(x, axis=1)))
  model.add(keras.layers.Dense(8, activation='softmax'))

  model.compile(optimizer = opt, loss='categorical_crossentropy',metrics = ['accuracy', keras.metrics.TruePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives()]) 
  model.summary()  
  model.fit(x = X_train, validation_data = test_set, epochs = 50)

# Training Custom CNN 2

In [110]:
def trainCustomCNN2(train_set, test_set):  #Ahmet şablon.
  X_train  = train_set 
  model = keras.Sequential()
  # LFLB1
  model.add(keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', data_format='channels_last',input_shape=(128,128,3)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('elu'))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  model.add(keras.layers.Dropout(0.5))

  model.add(keras.layers.Conv2D(filters=128,kernel_size=(3,3), strides=(1,1), padding='same'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('elu'))
  model.add(keras.layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))
  model.add(keras.layers.Dropout(0.25))

  model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('elu'))
  model.add(keras.layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))
  model.add(keras.layers.Dropout(0.25))

  model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Activation('elu'))
  model.add(keras.layers.MaxPooling2D(pool_size=(4,4), strides=(4,4)))
  model.add(keras.layers.Dropout(0.25))

  model.add(keras.layers.Flatten())
  # FC
  model.add(keras.layers.Dense(units=8, activation='softmax'))

  # Model compilation
  opt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9,  beta_2=0.999, amsgrad=False)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  model.summary()

  model.fit(x = X_train, validation_data = test_set, epochs = 500)

# Training Custom CNN 3

In [111]:
def trainCustomCNN3(train_set, test_set):
    X_train  = train_set
    opt  = tf.keras.optimizers.Adam(learning_rate=0.00005)

    #input_shape=(128, 128),
    model = keras.Sequential() 

    model.add(keras.layers.Conv2D(64, input_shape=(128, 128, 3), kernel_size=(3,3), strides=(1,1), padding='same', kernel_regularizer=keras.regularizers.l2(l=0.0005)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=4, strides=None, padding='same'))

    model.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=1, padding='same', kernel_regularizer=keras.regularizers.l2(l=0.0005)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=2, strides=None, padding='same'))

    model.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=1, padding='same', kernel_regularizer=keras.regularizers.l2(l=0.0005)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=2, strides=None, padding='same'))

    model.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=1, padding='same', kernel_regularizer=keras.regularizers.l2(l=0.0005)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.MaxPooling2D(pool_size=2, strides=None, padding='same'))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(8, activation='softmax'))

    model.compile(optimizer = opt, loss='categorical_crossentropy', metrics = ['accuracy', keras.metrics.FalsePositives()]) 
    model.summary()
    model.fit(x = X_train, validation_data = test_set, epochs = 100)
    model.save('/content/drive/My Drive/Bp_Models/MelSpectrogram_Mel128_AugDerivativeOrder2_v3')

    return model

# Training CNN With Transfer Learning

In [112]:
def transferLearning():  
  #Set pretrained Layers
  #for layer in model.layers[:150]:
  #    layer.trainable=False
  #for layer in model.layers[150:]:
  #    layer.trainable=True

  #opt = tf.keras.optimizers.SGD(learning_rate=0.001)


  opt  = tf.keras.optimizers.Adam(learning_rate=0.00001)
  for j in range(3,4):
    foldNo = 'fold' + str(j + 1)
    test_set  = testPreprocess(foldNo)
    train_set = trainPreprocess(foldNo)
    print("")
    print(foldNo)
    
    epochNo = 100     
    base_model = tf.keras.applications.ResNet101(
    include_top=False, weights='imagenet', classes=3)
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    preds=Dense(3,activation='softmax')(x) #final layer with softmax activation
    model=Model(inputs=base_model.input,outputs=preds)


    print("Epoch: " + str(epochNo) + "\n\n")
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalsePositives(), keras.metrics.FalseNegatives()])
    model.fit(x = train_set, validation_data = test_set, epochs = epochNo)
    print("Egitim basariyla tamamlandi.")

# Machine Learning

In [113]:
def compareLists(l1, l2):
  if len(l1) != len(l2):
    print('Lengths are not equal!')

  check = True  
  for i in range(len(l1)):
    if l1[i] != l2[i]:
      check = False
      break
  return check

def getCategoryValue(x):
  yList =[[1. ,  0. ,  0. , 0. ,  0. ,  0. ,  0. , 0.],
          [0. ,  1. ,  0. , 0. ,  0. ,  0. ,  0. , 0.],
          [0. ,  0. ,  1. , 0. ,  0. ,  0. ,  0. , 0.],
          [0. ,  0. ,  0. , 1. ,  0. ,  0. ,  0. , 0.],
          [0. ,  0. ,  0. , 0. ,  1. ,  0. ,  0. , 0.],
          [0. ,  0. ,  0. , 0. ,  0. ,  1. ,  0. , 0.],
          [0. ,  0. ,  0. , 0. ,  0. ,  0. ,  1. , 0.],
          [0. ,  0. ,  0. , 0. ,  0. ,  0. ,  0. , 1.]]


  for i in range(len(yList)):
    if compareLists(x, yList[i]):
      return (i + 1)

  print("Bir değer döndürülemedi, hata var!")


def getDatasetForMachineLearning(train_set, test_set, trainOutput = 1, testOutput = 1):
  train_X = []
  train_Y = []
  test_X = []
  test_Y = []

  setTrainY = []
  setTestY = []

  y_doubles = []


  # for element in list(test_set.as_numpy_iterator()):
  #   for className in element[1]:




  for element in list(train_set.as_numpy_iterator()):
    for className in element[1]:
      train_Y.append(getCategoryValue(className))
    for features in element[0]:
      train_X.append(features)  
  for element in list(test_set.as_numpy_iterator()):
    for className in element[1]:
      test_Y.append(getCategoryValue(className))
    for features in element[0]:
      test_X.append(features)   



  train_Y = np.array(train_Y)
  test_Y = np.array(test_Y)
  train_X = np.array(train_X)
  test_X = np.array(test_X)

  return train_X, train_Y, test_X, test_Y


In [114]:
def classifyWithMachineLearning(train_set, test_set):
  from keras.models import Model
  from keras.models import load_model

  from sklearn.neural_network import MLPClassifier
  from sklearn.linear_model import LogisticRegression
  from sklearn.ensemble import RandomForestClassifier
  from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
  from sklearn.neighbors import KNeighborsClassifier
  from sklearn.svm import SVC
  from sklearn.naive_bayes import GaussianNB
  from sklearn.ensemble import GradientBoostingClassifier
  from sklearn.ensemble import AdaBoostClassifier
  from sklearn.svm import LinearSVC

  train_X, train_Y, test_X, test_Y = getDatasetForMachineLearning(train_set, test_set)



  random_state = 100
  cnnPath = '/content/drive/My Drive/Bp_Models/MelSpectrogramOriginal_Mel128_v3'

  cnn = load_model(cnnPath)
  outputLayer = cnn.layers[-5] #Dense layer 512.
  print(outputLayer.output.name)

  

  intermediate_layer_model = Model(inputs=cnn.input,
                                  outputs=outputLayer.output)
  
  print("\nTrain Set:")
  train_output = intermediate_layer_model.predict(train_X)
  #print(train_output)
  print("Boyut1: " + str(len(train_output)))
  print("Boyut2: " + str(len(train_output[0])))
  train_output = np.array(train_output)



  print("\nTest Set:")
  test_output = intermediate_layer_model.predict(test_X)
  #print(test_output)
  print("Boyut1: " + str(len(test_output)))
  print("Boyut2: " + str(len(test_output[0])))
  test_output = np.array(test_output)
  
  models=[MLPClassifier(max_iter=200000, activation='tanh',solver='lbfgs', random_state=random_state),
          LogisticRegression(dual=False,multi_class='auto',solver='lbfgs',random_state=random_state),
          RandomForestClassifier(n_estimators=100,criterion='entropy'),
          LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto'),
          KNeighborsClassifier(n_neighbors=10, weights='distance'),
          SVC(kernel='poly',degree=2,C=100, gamma='auto'),
          GaussianNB(),
          GradientBoostingClassifier(),
          AdaBoostClassifier(),
          LinearSVC(penalty='l1',dual=False,multi_class='crammer_singer',max_iter=1000000),
          SVC(kernel='rbf', random_state=0, gamma=.01, C=100000)]



  # model = models[4].fit(train_output, train_Y)   
  # model.predict(test_output)
  # result = model.score(test_output, test_Y)  
  # result2 = model.score(train_output, train_Y)      
  # print("Test Set Accuracy: " + str(result))
  # print("Train Set Accuracy: " + str(result2))





  for i in range(len(models)):
    print("\n\nModel" + str(i) + ":" + str(models[i]))
    model = models[i].fit(train_output, train_Y)  # Bad input shape: 1, 3, 5, 6, 7, 8, 9, 10
    model.predict(test_output)

    testResult = model.score(test_output, test_Y)  
    trainResult = model.score(train_output, train_Y) 

    print("Test Set Accuracy: " + str(testResult))
    print("Train Set Accuracy: " + str(trainResult))


# Main

In [115]:
train_set, test_set = getDataset()
#model = trainCustomCNN3(train_set, test_set)
# print((list(train_set.as_numpy_iterator())[35][1]))
classifyWithMachineLearning(train_set, test_set)

######################### SEEDLER AYNI VERILMELI #####################


Found 1441 files belonging to 8 classes.
Using 1153 files for training.
Found 1441 files belonging to 8 classes.
Using 288 files for validation.
dense/BiasAdd:0

Train Set:
Boyut1: 1153
Boyut2: 512

Test Set:
Boyut1: 288
Boyut2: 512


Model0:MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=100, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
Test Set Accuracy: 0.6215277777777778
Train Set Accuracy: 1.0


Model1:LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
            

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Test Set Accuracy: 0.7118055555555556
Train Set Accuracy: 1.0


Model2:RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Test Set Accuracy: 0.6458333333333334
Train Set Accuracy: 1.0


Model3:LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage='auto',
                           solver='eigen', store_covariance=False, tol=0.0001)
Test Set Accuracy: 0.7013888888888888
Train Set Accuracy: 1.0


Model4:KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
        

# Making Single Prediction

In [116]:
!nvidia-smi

"""
img_path = '/content/drive/My Drive/Hushem set1/fold1/train/02_Tapered/image_053.BMP'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
"""
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)

Sat May 29 11:33:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    31W /  70W |   3408MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

"\nimg_path = '/content/drive/My Drive/Hushem set1/fold1/train/02_Tapered/image_053.BMP'\nimg = image.load_img(img_path, target_size=(224, 224))\nx = image.img_to_array(img)\nx = np.expand_dims(x, axis=0)\nx = preprocess_input(x)\npreds = model.predict(x)\n"

In [117]:
"""
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/My Drive/Set1_5xAugmented/fold3/test/04_Amorphous/image_032.BMP', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.argmax(result))
print(train_set.class_names)
"""

"\nimport numpy as np\nfrom keras.preprocessing import image\ntest_image = image.load_img('/content/drive/My Drive/Set1_5xAugmented/fold3/test/04_Amorphous/image_032.BMP', target_size = (64,64))\ntest_image = image.img_to_array(test_image)\ntest_image = np.expand_dims(test_image, axis = 0)\nresult = model.predict(test_image)\nprint(np.argmax(result))\nprint(train_set.class_names)\n"